In [1]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path

# Override/set credentials in env var
os.environ['CWD'] = str(Path(os.getcwd()).parent)

# Base paths
cwd = Path(os.environ['CWD'])
dir_data = cwd / 'data'

# Set mlflow artifacts location
dir_exp_root = Path(cwd / 'experiments')
dir_exp_this = Path(dir_exp_root / 'record_linkage')
dir_exp_mlflow = dir_exp_this / 'mlflow'
dir_exp_mlflow.mkdir(exist_ok=True, parents=True)

import mlflow
mlflow_tracking_uri = (dir_exp_mlflow / 'mlruns').as_uri()
print(f'mlflow URI: {mlflow_tracking_uri}')
mlflow.set_tracking_uri(mlflow_tracking_uri)

import numpy as np
from PIL import Image

mlflow URI: file:///root/work/research/experiments/record_linkage/mlflow/mlruns


In [2]:
from faker import Faker
Faker.seed(288)
fake = Faker(
#     [
#     'it_IT',
#     'en_US',
#     'es_ES',
#     'en_CA'
#     ]
)
import pandas as pd

db1 = pd.DataFrame([fake.profile() for _ in range(20000)])

In [3]:
# Add current_location
# Remove fields randomly 
# db1.head(5)

In [4]:
P_NOISE_CHAR = 0.1
NUM_TOKENS_KEPT_ADDR = 2
ROW_COL_MISSING_OR_SWAPPED = 'SWAP' # None, 'MISSING'
P_ROW_COL_MISSING_OR_SWAPPED = 1
FRAC_KEPT_ROWS_DB2 = 0.5


TFIDF_ANALYZER = 'char_wb'
TFIDF_NGRAM_LO = 2
TFIDF_NGRAM_HI = 4
TFIDF_MAX_DF = 0.8
TFIDF_MIN_DF = 10
TFIDF_MAX_FEATS = 100000

In [5]:
pd.options.mode.chained_assignment = None
np.random.seed(288)

db2 = db1[['job', 'address', 'name']]

import re
def _split(txt):
    return [x.strip() for x in re.split('-|\s| and |,', txt) if x.strip() != '']

# _split('Wilson, Sanchez and Pearson')

db2['job'] = db2['job'].apply(lambda txt: 
    np.random.choice(_split(txt))
)

db2['name'] = db2['name'].apply(lambda txt: 
    np.random.choice(_split(txt))
)



db2['address'] = db2['address'].apply(lambda txt: 
    ' '.join(np.random.choice(_split(txt), size=min(NUM_TOKENS_KEPT_ADDR, len(_split(txt))), replace=False)) # Keep 2 tokens when possible, otherwise 1
)

import string

def _add_char_noise(txt, p, noise_set_chars=[c for c in string.ascii_lowercase + string.digits]):
    txt_noise = ''
    for c in txt:
        if np.random.rand() < p:
            txt_noise += np.random.choice(noise_set_chars)
        else:
            txt_noise += c
    return txt_noise


from functools import partial

f_add_char_noise = partial(_add_char_noise, p=P_NOISE_CHAR)



db2['job'] = db2['job'].apply(f_add_char_noise)
db2['address'] = db2['address'].apply(f_add_char_noise)
db2['name'] = db2['name'].apply(f_add_char_noise)


_prime = '′'
d_name_map_c1c2 = {k:k+_prime for k in ['name', 'address', 'job']}
db2 = db2.rename(columns=d_name_map_c1c2)

In [6]:
def _random_swap_columns(row, subset, p):
    c1, c2 = np.random.choice(subset, 2, replace=False)
    aux = row[c1]
    row[c1] = row[c2]
    row[c2] = aux
    return row

In [7]:
def _random_set_empty_column(row, subset, p):
    c = np.random.choice(subset)
    row[c] = ''
    return row

In [8]:
rows_db2 = []

for _,row in db2.iterrows():
    if ROW_COL_MISSING_OR_SWAPPED == 'SWAP':
        row = _random_swap_columns(row,
                                   subset=list(d_name_map_c1c2.values()), # ['name′', 'address′', 'job′']
                                   p=P_ROW_COL_MISSING_OR_SWAPPED)
    elif ROW_COL_MISSING_OR_SWAPPED == 'MISSING':
        row = _random_set_empty_column(row,
                                       subset=list(d_name_map_c1c2.values()), #  ['name′', 'address′', 'job′']
                                       p=P_ROW_COL_MISSING_OR_SWAPPED)
    elif ROW_COL_MISSING_OR_SWAPPED is None:
        pass
    else:
        raise ValueError()
    rows_db2.append(row)
    
db2 = pd.DataFrame(rows_db2)

In [9]:
with pd.option_context('max.colwidth', None):
    display(db1[['name', 'address', 'job']].head())

,name,address,job
0,Amanda Arroyo,"0751 Samantha Walk Apt. 650\nChenfort, SC 04033",Herbalist
1,Victoria Brown,"787 Alexander Road\nPort Leslieborough, VA 53325",Outdoor activities/education manager
2,Amy Henry,"706 Sarah Lakes Apt. 421\nSouth Jeremy, AR 49313","Chemist, analytical"
3,Christopher Curtis,"7274 Bird Canyon Suite 720\nValentinechester, SC 19114","Geneticist, molecular"
4,Kevin Vargas,"1310 Anderson Fork Apt. 598\nBrandonbury, WV 31931",Structural engineer


In [10]:
db2[d_name_map_c1c2.values()].head()

,name′,address′,job′
0,Amanda,Herbasist,0751 Samantha
1,Leslieborsugh Alexander,Brotn,manaaer
2,6nalytical,SarahyS8uth,Henry
3,7274 SC,Curtis,Genetihdst
4,engineer,Apt. WV,Vargps


In [11]:
# missing rows + shuffle

db2 = db2.sample(frac=FRAC_KEPT_ROWS_DB2, random_state=288).reset_index()

In [12]:
# Toni: reduce dim

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer=TFIDF_ANALYZER,
                        ngram_range=(TFIDF_NGRAM_LO, TFIDF_NGRAM_HI),
                        max_df=TFIDF_MAX_DF,
                        min_df=TFIDF_MIN_DF,
                        max_features=TFIDF_MAX_FEATS)

In [13]:
X = np.concatenate([
    [str(x) for x in db1.to_numpy().flatten()],
    [str(x) for x in db2.to_numpy().flatten()]
])
tfidf.fit(X)
TFIDF_VOCAB = len(tfidf.vocabulary_)
TFIDF_VOCAB

48173

In [14]:
# from sklearn.metrics.pairwise import cosine_similarity
# l_d_sim = []
# for c1 in db1.columns:
#     for c2 in db2.columns:
#         X1 = tfidf.transform([str(x) for x in db1[c1].values]).astype(np.float32) # cast to np.float32 to avoid mem issues in cosine_sim mat
#         X2 = tfidf.transform([str(x) for x in db2[c2].values]).astype(np.float32)
#         M = cosine_similarity(X1, X2)
#         max_sim_c1c2 = M.max(axis=1).mean()
#         max_sim_c2c1 = M.max(axis=0).mean()
#         max_sim_mean = np.mean([max_sim_c1c2, max_sim_c2c1])
#         l_d_sim.append({
#             'c1': c1,
#             'c2': c2,
#             'sim': max_sim_mean
#         })
# #         print(f'Similarity index between DB1({c1}) and DB2({c2}): {max_sim_c1c2}')
# df_sim = pd.DataFrame(l_d_sim).sort_values('sim', ascending=False)


# TO_MATCH = ["name'", "company'", "address'"]
# matched = set()
# rows_kept = []

# for _, row in df_sim.iterrows():
#     c1 = row['c1']
#     c2 = row['c2']
#     if (c1 in matched or c2 in matched)\
#     or (c1 not in TO_MATCH and c2 not in TO_MATCH):
#         continue
#     else:
#         matched = matched.union(set([c1, c2]))
#         rows_kept.append(row)


# df_c1c2 = pd.DataFrame(rows_kept)

# d_c1c2 = df_c1c2.set_index('c1')['c2'].to_dict()
# d_c1c2

# Rows matching

In [15]:
from scipy.sparse import vstack, hstack
def vectorize_df_ordered_cols(df):
    vectorized_tfidf = np.vectorize(lambda x: tfidf.transform([x]))
    db_mat = df.to_numpy()
    X_db = vectorized_tfidf(db_mat)
    X_db = [hstack(row) for row in X_db]
    X_db = vstack(X_db)
    return X_db

In [16]:
class Clock():
    def __init__(self):
        self.d = {}
        
    def tick(self):
        self.t = time()
        
    def tack(self, name):
        self.d[name] = round(time() - self.t, 2)

In [17]:
from time import time
clock = Clock()

clock.tick()
X_db1 = vectorize_df_ordered_cols(db1[d_name_map_c1c2.keys()]).astype(np.float32)
clock.tack('time_vectorize_db1')

clock.tick()
X_db2 = vectorize_df_ordered_cols(db2[d_name_map_c1c2.values()]).astype(np.float32)
clock.tack('time_vectorize_db2')

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

clock.tick()
M_rowsim = cosine_similarity(X_db1, X_db2)

matching_row_in_db2 = M_rowsim.argmax(axis=1)
matching_row_in_db2_sim = M_rowsim.max(axis=1)
clock.tack('time_matching')

db1['matching_row_in_db2'] = matching_row_in_db2
db1['sim'] = matching_row_in_db2_sim

In [19]:
clock.tick()
list_top5 = []
for row in M_rowsim:
    list_top5.append(row.argsort()[::-1][:5])
    
matching_row_in_db2_top5 = np.array(list_top5)
clock.tack('time_argsort_for_topk')

In [20]:
db1['matching_row_in_db2_top5'] = [row for row in matching_row_in_db2_top5]

In [21]:
# db1[list(d_c1c2.keys())+['matching_row_in_db2']]

In [22]:
# Slide: Not noisy vs correspondent noisy

In [23]:
# Ruido2: Falta algun campo
# Ruido3: Swap campo
# Rendimiento SCANN

In [24]:
df_row_matches = pd.merge(
    left=db1.reset_index()[['index', 'matching_row_in_db2', 'company', 'address', 'name', 'sim', 'matching_row_in_db2_top5']],
    right=db2.reset_index()[['level_0', 'index'] + list(d_name_map_c1c2.values())],
    left_on='index',
    right_on='index'
)

# Metrics

In [25]:
def df_to_html_file_for_mlflow(df, path_artifact):
    # Also, write HTML and log 
    from pretty_html_table import build_table
    with pd.option_context("display.precision", 4):
        html_df = build_table(df, index=True, color='grey_light', font_family='Arial', font_size=12)
        with open(path_artifact, 'w') as fb:
            fb.write(html_df)
            
            
dir_artifacts = Path('output') # Can it be temp?
dir_artifacts.mkdir(parents=True, exist_ok=True)

In [26]:
d_metrics = {}

In [27]:
# Top1 acc - no thr
df_row_matches['high_conf'] = df_row_matches['sim'] > 0

df_row_matches['match_correct'] = df_row_matches['matching_row_in_db2'] == df_row_matches['level_0']
df_ct = pd.crosstab(df_row_matches['high_conf'], df_row_matches['match_correct'], normalize='all')
d_metrics['top1 acc'] = df_ct[True][True]

In [28]:
# Top1 acc - thr
THR_CONFIDENCE_QUANTILE = 0.3 # Example of threshold threshold tests → We will not process bottom 30% confident rows
# THR LEAVING OUT % OF SAMPLES
THR = df_row_matches['sim'].quantile(THR_CONFIDENCE_QUANTILE)
df_row_matches['high_conf'] = df_row_matches['sim'] > THR
df_ct = pd.crosstab(df_row_matches['high_conf'], df_row_matches['match_correct'], normalize='index')
d_metrics[f'top1 acc of processed'] = df_ct[True][True]

In [29]:
# norm this by 'all' instead of 'index'
df_ct_for_barplot = pd.crosstab(df_row_matches['high_conf'], df_row_matches['match_correct'], normalize='all', dropna=False)

In [30]:
df_to_html_file_for_mlflow(df_ct.reset_index(), dir_artifacts / 'error_rate_thr_norm_row.html')

In [31]:
df_to_html_file_for_mlflow(df_ct_for_barplot.reset_index(), dir_artifacts / 'error_rate_thr_norm_all.html')

In [32]:
df_ct_for_barplot
d_metrics_thr = {
    'perc_processed_ok': df_ct_for_barplot[True][True],
    
    'perc_processed_ko': df_ct_for_barplot[False][True],
    
    'perc_not_processed': THR_CONFIDENCE_QUANTILE,
}
df_metrics_thr = pd.DataFrame(d_metrics_thr.items())

df_metrics_thr[' '] = ''
df_metrics_thr[1] = (df_metrics_thr[1]*100).apply(lambda x: round(x,2))

import plotly.express as px
fig = px.bar(
    df_metrics_thr,
    x=' ',
    y=1,
    color=0,
    color_discrete_map={
        'perc_not_processed': 'rgba(70,70,70,0.5)',
        'perc_processed_ok': 'rgba(0,200,120,1)',
        'perc_processed_ko': 'red'
    },
    text=1,
    width=400,
    
)

fig.update_layout(
    uniformtext_minsize=12,
    uniformtext_mode='hide',
    legend_title_text='Case',
    xaxis_title="",
    yaxis_title="%",
    title='Percentages of not processed, processed correctly and<br>processed incorrectly when using threshold',
)
fig.update_layout(title_font_size=12)

fig.write_html(dir_artifacts / 'perc_not_proc_proc_ok_ko.html')

In [33]:
# Top3 acc
df_row_matches['match_correct_top3'] = df_row_matches.apply(lambda row: row['level_0'] in row['matching_row_in_db2_top5'][:3], axis=1)
df_ct = pd.crosstab(df_row_matches['high_conf'], df_row_matches['match_correct_top3'], normalize='index')
d_metrics['top3 acc'] = df_ct[True][True]

In [34]:
# Top5 acc
df_row_matches['match_correct_top5'] = df_row_matches.apply(lambda row: row['level_0'] in row['matching_row_in_db2_top5'], axis=1)
df_ct = pd.crosstab(df_row_matches['high_conf'], df_row_matches['match_correct_top5'], normalize='index')
d_metrics['top5 acc'] = df_ct[True][True]

In [35]:
# !pip install scann

In [36]:
# !pip install numpy --upgrade

In [37]:
# # https://github.com/google-research/google-research/blob/master/scann/docs/example.ipynb
# import scann


In [38]:
# searcher = scann.scann_ops_pybind.builder(X_db1, 10, "dot_product").tree(
#     num_leaves=2000, num_leaves_to_search=100, training_sample_size=250000).score_ah(
#     2, anisotropic_quantization_threshold=0.2).reorder(100).build()

In [39]:
sample_rows = db1.sample(7)
with pd.option_context('max.colwidth', None):
    display(sample_rows[['residence']])

,residence
3897,"5405 Jessica Grove\nNorth Matthew, MD 73186"
15719,"3205 Bowman Isle Suite 454\nLake David, IN 66600"
17915,"354 Allen Fort Apt. 673\nHayestown, CA 53217"
12783,"92597 Annette Branch\nYoungchester, MO 29107"
6880,"4177 Sarah Pass\nJacksonview, ID 21177"
15785,"PSC 3693, Box 9492\nAPO AE 88261"
2426,"57520 Costa Hill Suite 261\nLake Julieborough, HI 65335"


In [40]:
# # Example libpostal
# from postal.parser import parse_address
# from postal.expand import expand_address
# db1_residences_parsed = sample_rows[['residence']]
# db1_residences_parsed['residence'] = db1_residences_parsed['residence'].apply(
#     lambda txt: dict([(v,k) for k,v in parse_address(txt)]))
# pd.json_normalize(db1_residences_parsed['residence'])

In [41]:
import plotly.express as px
fig = px.histogram(
    df_row_matches['sim'],
    color=df_row_matches['match_correct'],
    barmode='stack'
)
fig.add_vline(x=THR, line_dash='dash')
fig.update_layout(
    legend_title_text='Match correct',
    xaxis_title="Confidence",
    yaxis_title="Num matched elements",
    title='Error rate by confidence interval'
)
fig.write_html(dir_artifacts / 'error_rate_by_conf.html')

In [42]:
with mlflow.start_run():
    for k in [
        'NUM_TOKENS_KEPT_ADDR',
        'P_NOISE_CHAR',
        'ROW_COL_MISSING_OR_SWAPPED',
        'P_ROW_COL_MISSING_OR_SWAPPED',
        'FRAC_KEPT_ROWS_DB2',
        'TFIDF_ANALYZER',
        'TFIDF_NGRAM_LO',
        'TFIDF_NGRAM_HI',
        'TFIDF_MAX_DF',
        'TFIDF_MIN_DF',
        'TFIDF_MAX_FEATS',
        'TFIDF_VOCAB',
        'THR_CONFIDENCE_QUANTILE',
        'THR',
    ]:
        mlflow.log_param(k, locals()[k])
    
    for k, v in d_metrics.items():
        mlflow.log_metric(k, v)
        
    for k, v in clock.d.items():
        mlflow.log_metric(k, v)
    
    mlflow.log_artifacts(dir_artifacts)